In [4]:
import risktools as rt
from loader import _load_json
import pandas as pd
import os

In [5]:
up = {"m*": {"user": "", "pass": ""}, "eia": "", "quandl": ""}

up["eia"] = os.getenv("EIA")
up["quandl"] = os.getenv("QUANDL")
up["m*"]["pass"] = os.getenv("MS_PASS")
up["m*"]["user"] = os.getenv("MS_USER")


In [11]:
ac = _load_json("../pytest/eia2tidy1.json")
ac = ac[["date", "value"]].set_index("date").sort_index().value
ts = rt.get_eia_df("PET.MCRFPTX2.M", key=up["eia"])
ts = ts[["date", "value"]].set_index("date").sort_index().value
ts = ts[ac.index.min() : ac.index.max()]

# assert ac.equals(ts), "get_eia_df Test 1 failed"
pd.testing.assert_series_equal(ac, ts)


        series       date  value
0    TexasProd 2021-10-01   4906
1    TexasProd 2021-09-01   4950
2    TexasProd 2021-08-01   4865
3    TexasProd 2021-07-01   4812
4    TexasProd 2021-06-01   4782
..         ...        ...    ...
485  TexasProd 1981-05-01   2577
486  TexasProd 1981-04-01   2578
487  TexasProd 1981-03-01   2604
488  TexasProd 1981-02-01   2593
489  TexasProd 1981-01-01   2600

[490 rows x 3 columns]


In [12]:
ts

date
1981-01-01    2600
1981-02-01    2593
1981-03-01    2604
1981-04-01    2578
1981-05-01    2577
              ... 
2021-06-01    4782
2021-07-01    4812
2021-08-01    4865
2021-09-01    4950
2021-10-01    4906
Name: value, Length: 490, dtype: int64

In [58]:

ac = _load_json("eia2tidy2.json")
ac = ac.set_index(["series", "date"]).sort_index().value
ts = rt.get_eia_df(
    ["PET.W_EPC0_SAX_YCUOK_MBBL.W", "NG.NW2_EPG0_SWO_R48_BCF.W"], key=up["eia"]
)
ts.date = pd.to_datetime(ts.date)
ts.table_name = ts.table_name.map(
    {
        "Cushing, OK Ending Stocks excluding SPR of Crude Oil, Weekly": "CrudeCushing",
        "Weekly Lower 48 States Natural Gas Working Underground Storage, Weekly": "NGLower48",
    }
)

ts = (
    ts.rename({"table_name": "series"}, axis=1)
    .set_index(["series", "date"])
    .sort_index()
    .value
)


            series       date  value
0     CrudeCushing 2021-12-24  34729
1     CrudeCushing 2021-12-17  33674
2     CrudeCushing 2021-12-10  32211
3     CrudeCushing 2021-12-03  30917
4     CrudeCushing 2021-11-26  28544
...            ...        ...    ...
1546     NGLower48 2010-01-29   2406
1547     NGLower48 2010-01-22   2521
1548     NGLower48 2010-01-15   2607
1549     NGLower48 2010-01-08   2850
1550     NGLower48 2010-01-01   3117

[1551 rows x 3 columns]


In [59]:
cmin = ac['CrudeCushing'].index.min()
cmax = ac['CrudeCushing'].index.max()

nmin = ac['NGLower48'].index.min()
nmax = ac['NGLower48'].index.max()

In [60]:
ts = ts.loc[('CrudeCushing',slice(cmin,cmax))].append(ts.loc[('NGLower48',slice(nmin,nmax))])

In [69]:
pd.testing.assert_series_equal(ac, ts)